<a href="https://colab.research.google.com/github/yosshy2020/intro_git/blob/master/cnntrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install hyperas

In [3]:
from hyperopt import hp
from hyperopt import Trials, tpe
from hyperas import optim 
from hyperas.distributions import choice, uniform

In [4]:
def prepare_data():
  
  import numpy as np
  import pandas as pd
  from tensorflow.keras.utils import to_categorical
  from sklearn.model_selection import KFold
  from sklearn.preprocessing import MinMaxScaler
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense, Activation ,Dropout

  train = pd.read_csv('/content/drive/MyDrive/biginners/train.csv')
  test = pd.read_csv('/content/drive/MyDrive/biginners/test.csv')

  all_data = pd.concat((train.loc[:,'age':'native-country'],
                      test.loc[:,'age':'native-country']))
  
  all_data_df =  pd.get_dummies(all_data, columns=["workclass"])
  all_data_df =  pd.get_dummies(all_data_df, columns=["education"])
  all_data_df =  pd.get_dummies(all_data_df, columns=["marital-status"])
  all_data_df =  pd.get_dummies(all_data_df, columns=["occupation"])
  all_data_df =  pd.get_dummies(all_data_df, columns=["relationship"])
  all_data_df =  pd.get_dummies(all_data_df, columns=["race"])
  all_data_df =  pd.get_dummies(all_data_df, columns=["sex"], drop_first=True)
  all_data_df =  pd.get_dummies(all_data_df, columns=["native-country"])

  scaler = MinMaxScaler()
  all_data_df[['age', 'fnlwgt', 'education-num']] = scaler.fit_transform(all_data_df[['age', 'fnlwgt', 'education-num']])

  train_X = all_data_df[:train.shape[0]]
  test_X = all_data_df[train.shape[0]:]
  train_Y= train.Y

  kf = KFold(n_splits=4, shuffle= True, random_state=123)
  tr_idx, va_idx = list(kf.split(train_X))[0] 

  tr_x, va_x = train_X.iloc[tr_idx], train_X.iloc[va_idx]
  tr_y, va_y = train_Y.iloc[tr_idx], train_Y.iloc[va_idx]

  return tr_x, tr_y, va_x, va_y

In [5]:
def create_model(tr_x, tr_y):
  model = Sequential()

  model.add(Dense(
      {{choice([500, 784])}},
      input_dim=tr_x.shape[1],
      activation='relu'
       ))
  model.add(Dropout(0.4))

  if {{choice(['none', 'one', 'two'])}} == 'none':
    pass

  elif {{choice(['none', 'one', 'two'])}} == 'one':
    model.add(Dense(
      {{choice([100, 200])}},
      activation='relu'
      ))
  
  elif {{choice(['none', 'one', 'two'])}} == 'two':
    model.add(Dense(
      {{choice([100, 200])}},
      activation='relu'
      ))
    model.add(Dense(
      {{choice([25, 50])}},
      activation='relu'
      ))
    
  model.add(Dense(1, activation='sigmoid'))

  model.compile(
    loss='mean_squared_error',
    optimizer={{choice(['adam', 'rmsprop'])}},
    metrics=['accuracy'])
  
  epoch =10
  batch_size =100

  result = model.fit(tr_x, tr_y,
                   epochs=epoch,
                   batch_size=batch_size,
                   validation_data=(va_x, va_y),
                   verbose=0)
  
  validation_acc =np.amax(result.history['val_accuracy'])
  print('Accuracy in search', validation_acc)

  return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}
  

In [6]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=prepare_data,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          notebook_name = '/drive/MyDrive/Colab Notebooks/cnntrain',
                                          trials=Trials())

>>> Imports:
#coding=utf-8

try:
    from google.colab import drive
except:
    pass

try:
    from hyperopt import hp
except:
    pass

try:
    from hyperopt import Trials, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical
except:
    pass

try:
    from sklearn.model_selection import KFold
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Activation, Dropout
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [500, 784]),
        'Dense_1': hp.choice('Dense_1', ['none', 'one', 'two']),
        'Dense_2': hp.cho

In [7]:
print(best_model.summary())

print(best_run)


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 500)               28000     
_________________________________________________________________
dropout_17 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 501       
Total params: 28,501
Trainable params: 28,501
Non-trainable params: 0
_________________________________________________________________
None
{'Dense': 0, 'Dense_1': 1, 'Dense_2': 2, 'Dense_3': 1, 'Dense_4': 0, 'Dense_5': 1, 'Dense_6': 0, 'optimizer': 1}


In [8]:
_, _, va_x, va_y = prepare_data()
val_loss, val_acc = best_model.evaluate(va_x, va_y)
print("val_loss: ", val_loss)
print("val_acc", val_acc)

93/93 [==============================] - 0s 993us/step - loss: 0.1117 - accuracy: 0.8454
val_loss:  0.11169950664043427
val_acc 0.8453781604766846
